In [1]:
import os
import json
from data_loader.SingleDataLoader import SingleDataLoader
from data_loader.DoubleDataLoader import DoubleDataLoader
from models.LeNet import LeNet
from models.AlexNet import Alexnet
from models.ResNet import ResNet
from models.ConvNeXt import ConvNeXt3D
from trainer.train import train_func
from trainer.aug_train import aug_train_func
from lib.path import get_training_data_dir
from custom_losses.dice import dice_loss, dice_coefficient
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.losses import BinaryFocalCrossentropy, BinaryCrossentropy

2024-05-25 14:37:10.313476: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-25 14:37:10.315956: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-25 14:37:10.348611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 14:37:10.348633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 14:37:10.349792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
data_dir = '../data'
train_list = os.path.join(data_dir, 'train_list')
test_list = os.path.join(data_dir, 'test_list')
val_list = os.path.join(data_dir, 'val_list')

In [3]:
DATA_TYPE1 = 'gr'
DATA_TYPE2 = 'Protein'
DATA_VOXEL_NUM = 10
CLASSIFYING_RULE = 'WaterClassifyingRuleSurface'
LIGAND_POCKET_DEFINER = 'LigandPocketDefinerOriginal'
LIGAND_VOXEL_NUM = 8

training_data_dir1 = get_training_data_dir(DATA_TYPE1, DATA_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, LIGAND_VOXEL_NUM)
training_data_dir2 = get_training_data_dir(DATA_TYPE2, DATA_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, LIGAND_VOXEL_NUM)

In [4]:
data_loader = SingleDataLoader(training_data_dir1)

In [5]:
data_loader = DoubleDataLoader(training_data_dir1, training_data_dir2)

In [6]:
train_data, train_labels = data_loader.load_data(train_list)
test_data, test_labels = data_loader.load_data(test_list)
val_data, val_labels = data_loader.load_data(val_list)

22
22
24
24
7
7
Error processing 184l: No data found for 184l
5
5
1
1
6
6
Error processing 186l: No data found for 186l
6
6
Error processing 187l: No data found for 187l
5
5
1
1
21
21
19
19
15
15
3
3
19
19
22
22
14
14
23
23
11
11
14
14
31
31
30
30
20
20
15
15
18
18
16
16
9
9
8
8
10
10
6
6
6
6
7
7
12
12
16
16
9
9
4
4
8
8
6
6
8
8
7
7
8
8
7
7
19
19
17
17
14
14
17
17
18
18
12
12
18
18
13
13
17
17
15
15
18
18
12
12
18
18
15
15
17
17
16
16
18
18
12
12
13
13
15
15
14
14
18
18
14
14
14
14
15
15
15
15
16
16
14
14
14
14
16
16
15
15
16
16
15
15
17
17
16
16
13
13
18
18
14
14
16
16
18
18
14
14
15
15
16
16
12
12
18
18
13
13
31
31
25
25
5
5
10
10
25
25
15
15
13
13
20
20
25
25
17
17
18
18
18
18
11
11
16
16
16
16
12
12
7
7
7
7
30
30
20
20
32
32
28
28
15
15
18
18
12
12
13
13
10
10
12
12
15
15
12
12
11
11
7
7
9
9
14
14
3
3
3
3
13
13
11
11
11
11
12
12
2
2
6
6
4
4
4
4
7
7
8
8
11
11
7
7
9
9
7
7
6
6
10
10
10
10
10
10
9
9
7
7
10
10
10
10
14
14
8
8
11
11
10
10
10
10
9
9
10
10
9
9
11
11
10
10
9
9
8
8
17
17
16
1

2024-05-25 14:38:03.518747: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


29
29
24
24
15
15
9
9
28
28
17
17
19
19
23
23
17
17
10
10
1
1
6
6
26
26
23
23
20
20
22
22
23
23
15
15
11
11
17
17
11
11
17
17
27
27
21
21
28
28
24
24
31
31
21
21
8
8
14
14
27
27
23
23
27
27
22
22
28
28
21
21
24
24
21
21
27
27
27
27
6
6
4
4
5
5
3
3
25
25
23
23
23
23
23
23
22
22
23
23
30
30
22
22
30
30
24
24
30
30
26
26
27
27
29
29
5
5
4
4
6
6
3
3
4
4
13
13
6
6
10
10
17
17
13
13
27
27
24
24
21
21
30
30
11
11
15
15
25
25
22
22
27
27
24
24
23
23
24
24
15
15
10
10
26
26
20
20
23
23
22
22
26
26
25
25
25
25
21
21
20
20
18
18
23
23
23
23
26
26
21
21
33
33
24
24
31
31
22
22
31
31
19
19
31
31
20
20
28
28
21
21
20
20
18
18
24
24
22
22
26
26
24
24
21
21
31
31
22
22
14
14
26
26
16
16
23
23
21
21
14
14
19
19
12
12
18
18
17
17
19
19
8
8
8
8
25
25
19
19
29
29
21
21
22
22
20
20
24
24
19
19
10
10
15
15
23
23
25
25
31
31
23
23
18
18
14
14
19
19
14
14
26
26
21
21
6
6
5
5
8
8
13
13
24
24
21
21
23
23
18
18
9
9
14
14
22
22
21
21
27
27
18
18
12
12
23
23
14
14
18
18
11
11
24
24
15
15
24
24
23
23
22
22
14
14
18

In [8]:
print('Train data shape: ', train_data.shape)
print('Train labels shape: ', train_labels.shape)
print('Test data shape: ', test_data.shape)
print('Test labels shape: ', test_labels.shape)
print('Val data shape: ', val_data.shape)
print('Val labels shape: ', val_labels.shape)

Train data shape:  (50275, 21, 21, 21, 6)
Train labels shape:  (50275,)
Test data shape:  (13510, 21, 21, 21, 6)
Test labels shape:  (13510,)
Val data shape:  (10317, 21, 21, 21, 6)
Val labels shape:  (10317,)


In [9]:
epochs = 1
batch_size = 200
input_shape = (DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, train_data.shape[-1])
n_base = 8
learning_rate = 1e-4
early_stopping = 300
BN = True
dropout = 0.5
model_func = ResNet
model_name = model_func.__name__
losses = [BinaryCrossentropy(label_smoothing=0.1), dice_loss]
loss= losses[0]
metrics = ['accuracy', dice_coefficient, Recall(), Precision()]
# path_type = '/'.join(training_data_dir1.split('/')[6:11])
path_type = 'test'
checkpoint_path = f"./checkpoints/{path_type}/{model_name}/" + "cp-{epoch:04d}.weights.h5"
model_checkpoint = True

In [10]:
pos = train_labels.sum()
neg = train_labels.shape[0] - pos
total = train_labels.shape[0]

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 1.0425306901128069, 1: 0.9608034246837136}


In [12]:
clf, clf_hist = aug_train_func(
                                x_train=train_data,
                                y_train=train_labels,
                                x_val=val_data,
                                y_val=val_labels,
                                input_shape=input_shape,
                                model_func=model_func,
                                loss=loss,
                                metrics=metrics,
                                epochs=epochs,
                                batch_size=batch_size,
                                num_rotations=1,
                                angle_unit=45,
                                n_base=n_base,
                                learning_rate=learning_rate,
                                early_stopping=early_stopping,
                                checkpoint_path=checkpoint_path,
                                model_checkpoint=model_checkpoint,
                                class_weight=class_weight,
                                BN = BN,
                                dropout=dropout
                            )

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 21, 21, 21, 6)]      0         []                            
                                                                                                  
 conv3d_9 (Conv3D)           (None, 21, 21, 21, 8)        1304      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, 21, 21, 21, 8)        32        ['conv3d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_7 (Activation)   (None, 21, 21, 21, 8)        0         ['batch_normalization_9[

KeyboardInterrupt: 

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,

: 

In [ ]:
# clf, clf_hist, clf_eval = train_func(
#                                     x_train=train_data,
#                                     y_train=train_labels,
#                                     x_test=test_data,
#                                     y_test=test_labels,
#                                     x_val=val_data,
#                                     y_val=val_labels,
#                                     input_shape=input_shape,
#                                     model_func=model_func,
#                                     loss=loss,
#                                     metrics=metrics,
#                                     epochs=epochs,
#                                     batch_size=batch_size,
#                                     n_base=n_base,
#                                     learning_rate=learning_rate,
#                                     early_stopping=early_stopping,
#                                     checkpoint_path=checkpoint_path,
#                                     model_checkpoint=model_checkpoint,
#                                     class_weight=class_weight,
#                                     BN = BN,
#                                     dropout=dropout
#                                 )

In [ ]:
prediction = clf.predict(test_data)

In [ ]:
prediction.round().sum()